In [12]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [73]:
#generating URLs for the first five pages
base_url = "https://www.seek.com.au/healthcare-data-analyst-jobs"
page_urls = [base_url]
for p in range(2,6):
    page_urls.append(f"{base_url}?page={p}")
print(page_urls)

#dataframe to store extracted info from jobs 
columns = ['Title', 'Company', 'Location', 'Short Description', 'Link']
job_data = pd.DataFrame(columns=columns)

['https://www.seek.com.au/healthcare-data-analyst-jobs', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=2', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=3', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=4', 'https://www.seek.com.au/healthcare-data-analyst-jobs?page=5']


In [75]:
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)

for url in page_urls:
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    #finding all the job cards
    job_cards = soup.find_all("article", {"data-automation" : "normalJob"})
    
    for card in job_cards:
        title = (card.find("a", {"data-automation" : "jobTitle"})).get_text(strip=True)
        company = (card.find("a", {"data-automation" : "jobCompany"})).get_text(strip=True)
        location = (card.find("a", {"data-automation" : "jobLocation"})).get_text(strip=True)
        short_desc = (card.find("span", {"data-automation" : "jobShortDescription"})).get_text(strip=True)
        title_link = card.find("a", {"data-automation" : "jobTitle"})["href"]
        full_link = "https://www.seek.com.au" + title_link
        
        job_data.loc[len(job_data)] = [title, company, location, short_desc, full_link]

driver.quit()

In [77]:
# Define your target folder
folder_path = "C:\\Users\\lubab\\OneDrive\\Documents\\Data Science Portfolio Projects\\Healthcare Data Anayst Jobs Australia\\healthcare-data-analyst-jobs-australia\\data\\raw"

# Make sure the folder exists (optional but recommended)
os.makedirs(folder_path, exist_ok=True)

# Full file path
file_path = os.path.join(folder_path, "healthcare_jobs.csv")

# Save CSV
job_data.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"Data saved to {file_path}")

Data saved to C:\Users\lubab\OneDrive\Documents\Data Science Portfolio Projects\Healthcare Data Anayst Jobs Australia\healthcare-data-analyst-jobs-australia\data\raw\healthcare_jobs.csv


## Extracting full description for each job

In [3]:
df = pd.read_csv("C:\\Users\\lubab\\OneDrive\\Documents\\Data Science Portfolio Projects\\Healthcare Data Anayst Jobs Australia\\healthcare-data-analyst-jobs-australia\\data\\raw\\healthcare_jobs.csv")
df.head(5)

,Title,Company,Location,Short Description,Link
0,Data Analyst,Western Health,St Albans,A unique opportunity exists to join the Wester...,https://www.seek.com.au/job/86244018?type=stan...
1,Population Health Data Analyst,South Eastern Melbourne Primary Health Network,Heatherton,Analyse data to drive population health planning,https://www.seek.com.au/job/86312389?type=stan...
2,Data Analyst,Western Health,Williamstown,Drive data-driven insights and improve organis...,https://www.seek.com.au/job/86238954?type=stan...
3,Data Analyst,Western Health,Footscray,"Excellent data management and problem skills, ...",https://www.seek.com.au/job/86244700?type=stan...
4,Senior Analyst,Alfred Health,Melbourne VIC,Permanent Full Time (1.0 EFT / 80 Hours) + ADO...,https://www.seek.com.au/job/86301836?type=stan...


In [22]:
#list of job ad URLs to loop through
jobAd_urls = df['Link'].tolist()

#list of full descriptions
full_descriptions = []

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)

for url in jobAd_urls:
    try:
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        #extracting the full descriptions
        full_desc = soup.find("div", {"data-automation": "jobAdDetails"})

        if full_desc:
            details = full_desc.get_text(separator="\n", strip=True)
        else:
            details = "Job description not found or job no longer advertised"

    except Exception as e:
        # Catch any unexpected error and store a placeholder
        print(f"Error with link {link}: {e}")
        details = "Error loading job page"
    
    
    full_descriptions.append(details)


driver.quit()
df["full description"] = full_descriptions

In [26]:
# Define your target folder
folder_path = "C:\\Users\\lubab\\OneDrive\\Documents\\Data Science Portfolio Projects\\Healthcare Data Anayst Jobs Australia\\healthcare-data-analyst-jobs-australia\\data\\raw"

# Make sure the folder exists (optional but recommended)
os.makedirs(folder_path, exist_ok=True)

# Full file path
file_path = os.path.join(folder_path, "healthcare_jobs_full.csv")

# Save CSV
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print(f"Data saved to {file_path}")

Data saved to C:\Users\lubab\OneDrive\Documents\Data Science Portfolio Projects\Healthcare Data Anayst Jobs Australia\healthcare-data-analyst-jobs-australia\data\raw\healthcare_jobs_full.csv
